In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf
import pandas as pd

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pathlib

from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split


In [2]:
os.listdir("../../../../../dataset/all/")
datapath = "../../../../../dataset/all/"

In [3]:
FAST_RUN = False
IMAGE_WIDTH=256
IMAGE_HEIGHT=256
IMAGE_SIZE=(IMAGE_WIDTH, IMAGE_HEIGHT)
IMAGE_CHANNELS=3

In [7]:
normal_history = []
gabor_history = []
# Importing the Keras libraries and packages
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Activation
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers

from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import time
import math
import matplotlib.pyplot as plt

from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

from sklearn.preprocessing import MinMaxScaler

for i in range(10):
    filenames = os.listdir(datapath)
    categories = []
    for filename in filenames:
        category = filename.split('.')[0]
        if category == 'dog':
            categories.append(1)
        else:
            categories.append(0)

    df = pd.DataFrame({
        'filename': filenames,
        'category': categories
    })
    df["category"] = df["category"].replace({0: 'cat', 1: 'dog'}) 
    train_df, validate_df = train_test_split(df, test_size=0.20) #, random_state=42)
    train_df = train_df.reset_index(drop=True)
    validate_df = validate_df.reset_index(drop=True)
    total_train = train_df.shape[0]
    total_validate = validate_df.shape[0]
    batch_size=32
    train_datagen = None
    train_datagen = ImageDataGenerator(
        rescale=1./255,
    )
    train_generator = None
    train_generator = train_datagen.flow_from_dataframe(
        train_df, 
        datapath, 
        x_col='filename',
        y_col='category',
        target_size=IMAGE_SIZE,
        class_mode='categorical',
        batch_size=batch_size
    )
    validation_datagen = None
    validation_datagen = ImageDataGenerator(rescale=1./255)
    validation_generator = None
    validation_generator = validation_datagen.flow_from_dataframe(
        validate_df, 
        datapath, 
        x_col='filename',
        y_col='category',
        target_size=IMAGE_SIZE,
        class_mode='categorical',
        batch_size=batch_size
    )
    num_classes = 2
    classifier = None
    classifier = Sequential([
        # layers.experimental.preprocessing.Rescaling(1./255, input_shape=(img_size, img_size, 3)),
        layers.Conv2D(32, kernel_size=(15,15), strides=(1,1), activation='relu', input_shape=(256, 256, 3)),
        layers.MaxPooling2D(pool_size=(2,2)),
        Dropout(0.5),
        layers.Conv2D(64, kernel_size=(3,3), strides=(2,2), activation='relu'),
        layers.MaxPooling2D(pool_size=(2,2)),
        Dropout(0.5),
        layers.Conv2D(128, kernel_size=(3,3), strides=(2,2), activation='relu'),
        layers.MaxPooling2D(pool_size=(2,2)),
        Dropout(0.5),
        layers.Conv2D(128, kernel_size=(3,3), strides=(2,2), activation='relu'),
        layers.MaxPooling2D(pool_size=(2,2)),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        Dropout(0.5),
        layers.Dense(128, activation='relu'),
        Dropout(0.5),
        layers.Dense(num_classes)
    ])
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10,  
                                  min_delta=1e-4, mode='min', verbose=1)
    stop_alg = EarlyStopping(monitor='val_loss', patience=35, 
                             restore_best_weights=True, verbose=1)
    callbacks = [stop_alg, reduce_lr]
    epochs=20 if FAST_RUN else 1000
    start = time.perf_counter()
    classifier.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    hist = None
    hist = classifier.fit(
        train_generator, 
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=total_validate//batch_size,
        steps_per_epoch=total_train//batch_size,
        callbacks=callbacks
    )
    finish = time.perf_counter()
    print("Normal learning iteration {} : start-{} finish-{} training time-{}".format(i,start, finish, finish-start))
    
    normal_history.append(hist)
    fig = plt.figure(figsize=(10,6))
    plt.plot(hist.history['loss'], color='#785ef0')
    plt.plot(hist.history['val_loss'], color='#dc267f')
    plt.title('Model Loss Progress')
    plt.ylabel('Binary Cross-Entropy Loss')
    plt.xlabel('Epoch')
    plt.legend(['Training Set', 'Test Set'], loc='upper right')
    plt.savefig('Iteration-{}-normal-loss.png'.format(i), dpi=350, bbox_inches='tight')
    plt.show()

    fig = plt.figure(figsize=(10,6))
    plt.plot(hist.history['accuracy'], color='#785ef0')
    plt.plot(hist.history['val_accuracy'], color='#dc267f')
    plt.title('Model Loss Progress')
    plt.ylabel('Binary Cross-Entropy Loss')
    plt.xlabel('Epoch')
    plt.legend(['Training Set', 'Test Set'], loc='upper right')
    plt.savefig('Iteration-{}-normal-accuracy.png'.format(i), dpi=350, bbox_inches='tight')
    plt.show()
    cnnl1 = classifier.layers[0].name   # get the name of the first conv layer
    W = classifier.get_layer(name=cnnl1).get_weights()[0]   #get the filters
    wshape = W.shape  #save the original shape
    # this part will scale to [0, 1] for visualization purposes
    scaler = MinMaxScaler()
    scaler.fit(W.reshape(-1,1))
    W = scaler.transform(W.reshape(-1,1))
    W = W.reshape(wshape)
    # since there are 32 filters, we will display them 8x4
    fig, axs = plt.subplots(8,4, figsize=(24,24))
    fig.subplots_adjust(hspace = .25, wspace=.001)
    axs = axs.ravel()
    for i in range(W.shape[-1]):
      # we reshape to a 3D (RGB) image shape and display
      h = np.reshape(W[:,:,:,i], (15,15,3))
      axs[i].imshow(h)
      axs[i].set_title('Filter ' + str(i))    
    plt.savefig('Iteration-{}-normal-trainedFilters.png'.format(i), bbox_inches='tight', dpi=350)

    
    num_classes = 2
    classifier = None
    classifier = Sequential([
        # layers.experimental.preprocessing.Rescaling(1./255, input_shape=(img_size, img_size, 3)),
        layers.Conv2D(32, kernel_size=(15,15), strides=(1,1), activation='relu', input_shape=(256, 256, 3)),
        layers.MaxPooling2D(pool_size=(2,2)),
        Dropout(0.5),
        layers.Conv2D(64, kernel_size=(3,3), strides=(2,2), activation='relu'),
        layers.MaxPooling2D(pool_size=(2,2)),
        Dropout(0.5),
        layers.Conv2D(128, kernel_size=(3,3), strides=(2,2), activation='relu'),
        layers.MaxPooling2D(pool_size=(2,2)),
        Dropout(0.5),
        layers.Conv2D(128, kernel_size=(3,3), strides=(2,2), activation='relu'),
        layers.MaxPooling2D(pool_size=(2,2)),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        Dropout(0.5),
        layers.Dense(128, activation='relu'),
        Dropout(0.5),
        layers.Dense(num_classes)
    ])
    def get_gabor_filters(inchannels, outchannels, kernel_size = (3,3)):
        delta = 1e-4
        freqs = (math.pi/2)*(math.sqrt(2)**(-np.random.randint(0,5, (outchannels, inchannels))))
        thetas = (math.pi/8)*np.random.randint(0,8, (outchannels, inchannels))
        sigmas = math.pi/(freqs)
        psis = math.pi * np.random.rand(outchannels, inchannels)
        x0, y0 = np.ceil(np.array(kernel_size)/2)    

        y, x = np.meshgrid(
                np.linspace(-x0 + 1, x0 + 0, kernel_size[0]),
                np.linspace(-y0 + 1, y0 + 0, kernel_size[1]),
        )
        filterbank = []

        for i in range(outchannels):
            for j in range(inchannels):
                freq = freqs[i][j]
                theta = thetas[i][j]
                sigma = sigmas[i][j]
                psi = psis[i][j]

                rotx = x * np.cos(theta) + y * np.sin(theta)
                roty = -x * np.sin(theta) + y * np.cos(theta)

                g = np.exp(
                    -0.5 * ((rotx ** 2 + roty ** 2) / (sigma + delta) ** 2)
                )
                g = g * np.cos(freq * rotx + psi)
    #             g = g / (2 * math.pi * (sigma ** 2))
    #             g = gabor_kernel(frequency=freq, bandwidth=sigma, theta=theta, offset=psi, n_stds=0).real
                filterbank.append(g)
        return filterbank

    filterbank = get_gabor_filters(3, 32, (15,15))

    fig = plt.subplots(8, len(filterbank)//8, figsize=(22,22))
    for i,gf in enumerate(filterbank):
        plt.subplot(8, len(filterbank)//8, i+1)
        plt.imshow(gf, cmap='gray')
        plt.axis('off')

    cnnl1 = classifier.layers[0].name   # get the name of the first conv layer
    W = classifier.get_layer(name=cnnl1).get_weights()[0]   #get the filters
    wshape = W.shape  #save the original shape
    gabor_filters = W
    for kernel_index in range(wshape[3]):
        for channel_index in range(3):
            gabor_filters[:,:,channel_index, kernel_index] = filterbank[kernel_index+channel_index]

    classifier.get_layer(name=cnnl1).set_weights([gabor_filters, classifier.get_layer(name=cnnl1).get_weights()[1]])
    filter_layers = []
    for i in range(32):
        for j in range(3):
            filter_layers.append(np.reshape(W[:,:,j, i], (15,15)))
    for i,gf in enumerate(filter_layers):
        plt.subplot(8, len(filterbank)//8, i+1)
        plt.imshow(gf, cmap='gray')
        plt.axis('off')

    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10,  
                                  min_delta=1e-4, mode='min', verbose=1)

    stop_alg = EarlyStopping(monitor='val_loss', patience=35, 
                             restore_best_weights=True, verbose=1)

    callbacks = [stop_alg, reduce_lr]

    epochs=20 if FAST_RUN else 1000

    start = time.perf_counter()
    classifier.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    hist = None
    hist = classifier.fit(
        train_generator, 
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=total_validate//batch_size,
        steps_per_epoch=total_train//batch_size,
        callbacks=callbacks
    )
    finish = time.perf_counter()
    print("Normal learning iteration {} : start-{} finish-{} training time-{}".format(i,start, finish, finish-start))

    gabor_history.append(hist)
    fig = plt.figure(figsize=(10,6))
    plt.plot(hist.history['loss'], color='#785ef0')
    plt.plot(hist.history['val_loss'], color='#dc267f')
    plt.title('Model Loss Progress')
    plt.ylabel('Binary Cross-Entropy Loss')
    plt.xlabel('Epoch')
    plt.legend(['Training Set', 'Test Set'], loc='upper right')
    plt.savefig('Iteration-{}-gabor-loss.png'.format(i), dpi=350, bbox_inches='tight')
    plt.show()

    fig = plt.figure(figsize=(10,6))
    plt.plot(hist.history['accuracy'], color='#785ef0')
    plt.plot(hist.history['val_accuracy'], color='#dc267f')
    plt.title('Model Loss Progress')
    plt.ylabel('Binary Cross-Entropy Loss')
    plt.xlabel('Epoch')
    plt.legend(['Training Set', 'Test Set'], loc='upper right')
    plt.savefig('Iteration-{}-gabor-gabor.png'.format(i), dpi=350, bbox_inches='tight')
    plt.show()

    cnnl1 = classifier.layers[0].name   # get the name of the first conv layer
    W = classifier.get_layer(name=cnnl1).get_weights()[0]   #get the filters
    wshape = W.shape  #save the original shape

    # this part will scale to [0, 1] for visualization purposes
    scaler = MinMaxScaler()
    scaler.fit(W.reshape(-1,1))
    W = scaler.transform(W.reshape(-1,1))
    W = W.reshape(wshape)

    # since there are 32 filters, we will display them 8x4
    fig, axs = plt.subplots(8,4, figsize=(24,24))
    fig.subplots_adjust(hspace = .25, wspace=.001)
    axs = axs.ravel()
    for i in range(W.shape[-1]):
      # we reshape to a 3D (RGB) image shape and display
      h = np.reshape(W[:,:,:,i], (15,15,3))
      axs[i].imshow(h)
      axs[i].set_title('Filter ' + str(i))    
    plt.savefig('Iteration-{}-gabor-trainedFilters.png'.format(i), bbox_inches='tight', dpi=350)

Found 20000 validated image filenames belonging to 2 classes.
Found 5000 validated image filenames belonging to 2 classes.
Epoch 1/1000
235/625 [==========>...................] - ETA: 29:11 - loss: 0.8600 - accuracy: 0.4944

KeyboardInterrupt: 